In [162]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import requests

In [163]:
#import data from csv file
df1 = pd.read_csv('S&P BSE Teck Historical Data.csv')

In [164]:
# API details (replace with actual values)
api_url = "https://api.mfapi.in/mf/120594"  # Replace with the actual API endpoint
api_params = {"scheme_code": 120594}  # Assuming scheme_code is the required parameter

# Make the API request
response = requests.get(api_url, params=api_params)
data = response.json()

# Create a DataFrame
df = pd.DataFrame(data["data"])  # Extract the "data" list from the response

# Add calculated column (replace with your calculation logic)
#df["calculated_column"] = df["nav"] * 1.1  # Example: increase NAV by 10%

# Save to CSV
df.to_csv("fund_data.csv", index=False)


In [165]:
# Rename the column first
df1.rename(columns={"Date": "date"}, inplace=True)

# Now convert the date format
df1["date"] = pd.to_datetime(df1["date"], format='%m/%d/%Y').dt.strftime("%d-%m-%Y")

In [166]:
print(df1.head())

         date      Price       Open       High        Low    Vol. Change %
0  20-01-2024  16,616.35  16,840.91  16,848.41  16,604.55     NaN   -0.87%
1  19-01-2024  16,762.62  16,673.91  16,779.41  16,673.79  59.58K    1.29%
2  18-01-2024  16,548.78  16,517.33  16,574.08  16,344.10  50.58K   -0.19%
3  17-01-2024  16,580.51  16,424.69  16,656.80  16,390.26  75.05K    0.22%
4  16-01-2024  16,544.56  16,740.20  16,740.20  16,501.08  73.74K   -1.12%


In [167]:
print(df1['date'].duplicated().sum())
print(df['date'].duplicated().sum())
print(df1['date'].isna().sum())
print(df['date'].isna().sum())
print(len(df1))
print(len(df))

0
0
0
0
2736
2719


In [168]:
#keep only the dates in df1 that are in df
df1 = df1[df1['date'].isin(df['date'])]
print(len(df1))

2719


In [169]:
print(df1.head())
print(df.head())
print(df1.tail())
print(df.tail())

         date      Price       Open       High        Low    Vol. Change %
1  19-01-2024  16,762.62  16,673.91  16,779.41  16,673.79  59.58K    1.29%
2  18-01-2024  16,548.78  16,517.33  16,574.08  16,344.10  50.58K   -0.19%
3  17-01-2024  16,580.51  16,424.69  16,656.80  16,390.26  75.05K    0.22%
4  16-01-2024  16,544.56  16,740.20  16,740.20  16,501.08  73.74K   -1.12%
5  15-01-2024  16,732.45  16,713.90  16,893.24  16,649.81  64.14K    1.79%
         date        nav
0  19-01-2024  196.09000
1  18-01-2024  193.29000
2  17-01-2024  194.08000
3  16-01-2024  193.88000
4  15-01-2024  195.54000
            date     Price      Open      High       Low    Vol. Change %
2731  08-01-2013  3,514.39  3,516.38  3,522.73  3,495.37   8.36K   -0.14%
2732  07-01-2013  3,519.49  3,524.77  3,537.07  3,514.18  10.46K    0.13%
2733  04-01-2013  3,514.91  3,502.28  3,517.65  3,498.69   8.84K    0.68%
2734  03-01-2013  3,491.06  3,453.27  3,496.87  3,448.30  12.69K    1.32%
2735  02-01-2013  3,445.46  3,

In [170]:
df1["Change %"] = df1["Change %"].str.replace("%", "")
#do it for each column except date
df1["Price"] = df1["Price"].str.replace(",", "")
df1["Open"] = df1["Open"].str.replace(",", "")
df1["High"] = df1["High"].str.replace(",", "")
df1["Low"] = df1["Low"].str.replace(",", "")
numeric_columns = df1.columns.difference(["date", "Vol."])

In [171]:
#convert Vol. to numeric and replace the values in the same column
#if K then multiply by 1000, if M then multiply by 1000000, if B then multiply by 1000000000
#create a function to do this
def adjust_volume(value):
    if pd.isna(value):
        return value  # Keep NaN as is

    if value.endswith("M"):
        return pd.to_numeric(value[:-1]) * 1000000
    elif value.endswith("B"):
        return pd.to_numeric(value[:-1]) * 1000000000
    elif value.endswith("K"):
        return pd.to_numeric(value[:-1]) * 1000
    else:
        return pd.to_numeric(value)  # No modifier, assume original value
df1['Vol.'] = df1['Vol.'].apply(adjust_volume)



In [172]:
print(df1.dtypes)

date         object
Price        object
Open         object
High         object
Low          object
Vol.        float64
Change %     object
dtype: object


In [173]:

#df1.dtypes
df1[numeric_columns] = df1[numeric_columns].apply(pd.to_numeric)
#df1[numeric_columns] = df1[numeric_columns].astype(float64)
df["nav"] = df["nav"].apply(pd.to_numeric)


In [174]:
merged_data = df.merge(df1, on="date")
print(merged_data.head())
print(merged_data.dtypes)

         date     nav     Price      Open      High       Low     Vol.  \
0  19-01-2024  196.09  16762.62  16673.91  16779.41  16673.79  59580.0   
1  18-01-2024  193.29  16548.78  16517.33  16574.08  16344.10  50580.0   
2  17-01-2024  194.08  16580.51  16424.69  16656.80  16390.26  75050.0   
3  16-01-2024  193.88  16544.56  16740.20  16740.20  16501.08  73740.0   
4  15-01-2024  195.54  16732.45  16713.90  16893.24  16649.81  64140.0   

   Change %  
0      1.29  
1     -0.19  
2      0.22  
3     -1.12  
4      1.79  
date         object
nav         float64
Price       float64
Open        float64
High        float64
Low         float64
Vol.        float64
Change %    float64
dtype: object


In [175]:
merged_data["mf_return"] = (merged_data["nav"] - merged_data["nav"].shift(1)) / merged_data["nav"].shift(1)
merged_data["index_return"] = (merged_data["Price"] - merged_data["Price"].shift(1)) / merged_data["Price"].shift(1)

In [176]:
# Calculate covariance and mean of excess return and benchmark return
cov_er_br = merged_data['mf_return'].cov(merged_data['index_return'])
mean_er = merged_data['mf_return'].mean()
mean_br = merged_data['index_return'].mean()

# Calculate beta (covariance of excess return and benchmark return / variance of benchmark return)
beta = cov_er_br / merged_data['index_return'].var()

# Calculate alpha (average excess return - beta * average benchmark return)
alpha = mean_er - beta * mean_br

# Add alpha and beta to the merged dataset
merged_data['alpha'] = alpha
merged_data['beta'] = beta

In [180]:
merged_data.head()

,date,nav,Price,Open,High,Low,Vol.,Change %,mf_return,index_return,alpha,beta
0,19-01-2024,196.09,16762.62,16673.91,16779.41,16673.79,59580.0,1.29,NaN,NaN,-0.000332,0.884402
1,18-01-2024,193.29,16548.78,16517.33,16574.08,16344.10,50580.0,-0.19,-0.014279,-0.012757,-0.000332,0.884402
2,17-01-2024,194.08,16580.51,16424.69,16656.80,16390.26,75050.0,0.22,0.004087,0.001917,-0.000332,0.884402
3,16-01-2024,193.88,16544.56,16740.20,16740.20,16501.08,73740.0,-1.12,-0.001031,-0.002168,-0.000332,0.884402
4,15-01-2024,195.54,16732.45,16713.90,16893.24,16649.81,64140.0,1.79,0.008562,0.011357,-0.000332,0.884402


In [181]:
merged_data.to_csv("merged_data.csv", index=False)